Shafee Syed-Quadri, Karthik Thiagarajan, Ammar Mustufa, 
Tom Starkie, Nawen Deng, Geer Zhang, Haoxiang Yi

In [1]:
pip install praw


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install --upgrade praw

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import praw
from requests import Session

reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx'
)




In [4]:
# Check if the authentication is successful by accessing the current Redditor
try:
    print(f"Authenticated as: {reddit.user.me()}")
except Exception as e:
    print(f"Failed to authenticate: {e}")


Authenticated as: wucanyue1


In [12]:
url = "https://www.reddit.com/r/mentalhealth/comments/1fzx6nt/i_need_a_rant_about_my_gf/"

submission = reddit.submission(url=url)

print(submission.title)
print(submission.score)
print(submission.num_comments)
print(submission.permalink)

# get all comments and who posted them
for comment in submission.comments:
    print(comment.body)
    print(comment.author)

# for each comment, get user name and storce in dataframe
comments = []
for comment in submission.comments:
    comments.append({'user': comment.author, 'comment': comment.body})

df = pd.DataFrame(comments)
print(df)


I need a rant about my gf 
29
45
/r/mentalhealth/comments/1fzx6nt/i_need_a_rant_about_my_gf/
Do not sacrifice your mental health and take verbal and emotional abuse just to not be alone. I will tell you tat does not end well 13 years for me and I was in psychiatric care 35 times for what it all did to me. It;s on HER and HER alone to handle her problems and mental health.
Key_Flounder_7149
U need drop her like yesterday.
At your age, u have your own mental and emotional development to focus on (I’m 46 M).

U don’t know who u are yet, your confidence will build with age.  No BS, it’s hard for men these days, and yeah you may need to put yourself out there again, but will build your character.

The last thing you should do is cling on to a toxic partner.
BigChemistry1322
You’re 16 chances are you’re not going to end up with her. Time to cut your loses and move on. If she does this now just imagine how much worse it’ll get down the line. Move on dude and fine someone better that respects 

In [5]:
# Fetch 5000 posts from the subreddit
url = "https://www.reddit.com/r/suicidewatch/"
subreddit = reddit.subreddit("suicidewatch")

# List to store post data
posts = []

# Loop through each post and gather the required details
for submission in subreddit.hot(limit=2000):
    posts.append({
        'title': submission.title,        # Title of the post
        'body': submission.selftext,      # Body (text) of the post
        'author': submission.author.name if submission.author else 'deleted'  # Author of the post
    })

# Create a DataFrame to store the collected data
df = pd.DataFrame(posts)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('suicidewatch.csv', index=False)

                                                 title  \
0    New wiki on how to avoid accidentally encourag...   
1    Please remember that NO ACTIVISM of any kind i...   
2                   I'm bleeding out on a park bench.    
3    Hiding information about how to kill oneself s...   
4    I am no longer enjoying life. If you are no lo...   
..                                                 ...   
944                                      I am a burden   
945                                Really struggling.    
946  Feels like the world is ending every day, alre...   
947  if it wasn’t for my belief in God I’d be dead ...   
948                             Grappling with forever   

                                                  body              author  
0    We've been seeing a worrying increase in pro-s...            SQLwitch  
1    Activism, i.e. advocating or fundraising for s...            SQLwitch  
2    There's an ambulance that keeps going up and d...  Rude-Geologist59

In [7]:
import praw
import pandas as pd
import time
import random
from prawcore.exceptions import RequestException, ResponseException

# Initialize the Reddit instance
reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx2043789012@'
)

# Fetch posts from different sorting categories and timeframes
subreddit = reddit.subreddit("mentalhealth")

# List to store post data
posts = []

# Set target post count between 1400 and 1600
desired_post_count = random.randint(1400, 1600)
batch_size = 2000

def fetch_posts(subreddit, sort_method, limit, time_filter=None):
    try:
        if sort_method == 'top' and time_filter:
            return subreddit.top(time_filter=time_filter, limit=limit)
        elif sort_method == 'hot':
            return subreddit.hot(limit=limit)
        elif sort_method == 'new':
            return subreddit.new(limit=limit)
    except (RequestException, ResponseException) as e:
        print(f"Hit rate limit or request error: {e}. Waiting 5 minutes before retrying...")
        time.sleep(300)  # Wait for 300 seconds (5 minutes) before retrying
        return fetch_posts(subreddit, sort_method, limit, time_filter)

# Function to show progress
def show_progress(collected_posts, target_posts):
    percentage = (collected_posts / target_posts) * 100
    print(f"Progress: {collected_posts}/{target_posts} posts collected ({percentage:.2f}% complete)")

# Fetch posts in batches until we reach the target range
while len(posts) < desired_post_count:
    # Fetch a batch of 100 posts from 'hot'
    for submission in fetch_posts(subreddit, 'hot', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id  # Add post ID to help identify duplicates
        })
        if len(posts) >= desired_post_count:
            break  # Stop once we reach the desired post count
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 posts from 'new'
    for submission in fetch_posts(subreddit, 'new', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
        if len(posts) >= desired_post_count:
            break
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'all time'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='all'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
        if len(posts) >= desired_post_count:
            break
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this year'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='year'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
        if len(posts) >= desired_post_count:
            break
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this month'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='month'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
        if len(posts) >= desired_post_count:
            break
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

# Only remove duplicates once we have collected enough posts
df = pd.DataFrame(posts)

# Remove duplicate posts based on the 'id' column
df.drop_duplicates(subset='id', keep='first', inplace=True)

# Drop the 'id' column since it's no longer needed
df.drop(columns=['id'], inplace=True)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('mentalhealth4.csv', index=False)


Progress: 997/1589 posts collected (62.74% complete)
Progress: 1589/1589 posts collected (100.00% complete)
Progress: 1590/1589 posts collected (100.06% complete)
Progress: 1591/1589 posts collected (100.13% complete)
Progress: 1592/1589 posts collected (100.19% complete)
                                                  title  \
0              r/MentalHealth is looking for moderators   
1                            Politics and Mental Health   
2     Be brutal, what would make you leave your frie...   
3     We'll be completely forgotten after a few gene...   
4                      My Place Is Right By Your Side 💕   
...                                                 ...   
994   How Does Childhood Abandonment Trauma Affect T...   
995                                         I love you🥺   
996                  I'm going insane I hate everything   
1589  Mental Health Awareness Month: I have schizoaf...   
1590       my boyfriend k*lled himself in my apartment    

                  

In [8]:
import praw
import pandas as pd
import time
import random
from prawcore.exceptions import RequestException, ResponseException

# Initialize the Reddit instance
reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx2043789012@'
)

# Fetch posts from different sorting categories and timeframes
subreddit = reddit.subreddit("mentalhealth")

# List to store post data
posts = []

# Set target post count between 1400 and 1600
desired_post_count = random.randint(1400, 1600)
batch_size = 1000

def fetch_posts(subreddit, sort_method, limit, time_filter=None):
    try:
        if sort_method == 'top' and time_filter:
            return subreddit.top(time_filter=time_filter, limit=limit)
        elif sort_method == 'hot':
            return subreddit.hot(limit=limit)
        elif sort_method == 'new':
            return subreddit.new(limit=limit)
    except (RequestException, ResponseException) as e:
        print(f"Hit rate limit or request error: {e}. Waiting 5 minutes before retrying...")
        time.sleep(300)  # Wait for 300 seconds (5 minutes) before retrying
        return fetch_posts(subreddit, sort_method, limit, time_filter)

# Function to show progress
def show_progress(collected_posts, target_posts):
    percentage = (collected_posts / target_posts) * 100
    print(f"Progress: {collected_posts}/{target_posts} posts collected ({percentage:.2f}% complete)")

# Fetch posts in batches until we reach the target range
while len(posts) < desired_post_count:
    # Fetch a batch of 100 posts from 'hot'
    for submission in fetch_posts(subreddit, 'hot', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id  # Add post ID to help identify duplicates
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 posts from 'new'
    for submission in fetch_posts(subreddit, 'new', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'all time'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='all'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this year'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='year'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this month'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='month'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this week'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='week'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'today'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='day'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

# Only remove duplicates once we have collected enough posts
df = pd.DataFrame(posts)

# Remove duplicate posts based on the 'id' column
df.drop_duplicates(subset='id', keep='first', inplace=True)

# Drop the 'id' column since it's no longer needed
df.drop(columns=['id'], inplace=True)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('mentalhealth.csv4', index=False)


Progress: 996/1417 posts collected (70.29% complete)
Progress: 1989/1417 posts collected (140.37% complete)
Progress: 2982/1417 posts collected (210.44% complete)
Progress: 3982/1417 posts collected (281.02% complete)
Progress: 4977/1417 posts collected (351.24% complete)
Progress: 5975/1417 posts collected (421.67% complete)
Progress: 6257/1417 posts collected (441.57% complete)
                                                  title  \
0              r/MentalHealth is looking for moderators   
1                            Politics and Mental Health   
2     Be brutal, what would make you leave your frie...   
3     We'll be completely forgotten after a few gene...   
4                      My Place Is Right By Your Side 💕   
...                                                 ...   
5641  Suggestions for self care while helping an eld...   
5642                       Need help understanding this   
5643                      Struggling with friend group    
5644  I haven’t been feelin

In [17]:
import praw
import pandas as pd
import time
import random
from datetime import datetime
from prawcore.exceptions import RequestException, ResponseException, BadRequest

# Initialize the Reddit instance
reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx2043789012@'
)

# Fetch posts from different sorting categories and timeframes
subreddit = reddit.subreddit("mentalhealth")

# List to store post data
posts = []

# Set target post count between 1400 and 1600
desired_post_count = random.randint(1400, 1600)
batch_size = 1000

def fetch_posts(subreddit, sort_method, limit, time_filter=None):
    try:
        if sort_method == 'top' and time_filter:
            return subreddit.top(time_filter=time_filter, limit=limit)
        elif sort_method == 'controversial' and time_filter:
            return subreddit.controversial(time_filter=time_filter, limit=limit)
        elif sort_method == 'hot':
            return subreddit.hot(limit=limit)
        elif sort_method == 'new':
            return subreddit.new(limit=limit)
        elif sort_method == 'rising':
            return subreddit.rising(limit=limit)
        elif sort_method == 'random_rising':
            return subreddit.random_rising(limit=limit)
    except (RequestException, ResponseException, BadRequest) as e:
        print(f"API Error ({e}). Retrying after a 5-minute wait...")
        time.sleep(300)  # Wait for 300 seconds (5 minutes) before retrying
        return fetch_posts(subreddit, sort_method, limit, time_filter)

def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# Function to show progress
def show_progress(collected_posts, target_posts):
    percentage = (collected_posts / target_posts) * 100
    print(f"Progress: {collected_posts}/{target_posts} posts collected ({percentage:.2f}% complete)")

# Fetch posts in batches until we reach the target range
while len(posts) < desired_post_count:
    # Fetch a batch of 100 posts from 'hot'
    for submission in fetch_posts(subreddit, 'hot', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id  # Add post ID to help identify duplicates
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 posts from 'new'
    for submission in fetch_posts(subreddit, 'new', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'all time'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='all'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this year'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='year'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this month'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='month'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this week'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='week'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'today'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='day'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 rising posts
    for submission in fetch_posts(subreddit, 'rising', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 controversial posts from 'this year'
    for submission in fetch_posts(subreddit, 'controversial', batch_size, time_filter='year'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 random rising posts (if supported)
    try:
        for submission in fetch_posts(subreddit, 'random_rising', batch_size):
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
        show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch
    except AttributeError:
        print("random_rising is not supported on this subreddit.")

# Only remove duplicates once we have collected enough posts
df = pd.DataFrame(posts)

# Remove duplicate posts based on the 'id' column
df.drop_duplicates(subset='id', keep='first', inplace=True)

# Drop the 'id' column since it's no longer needed
df.drop(columns=['id'], inplace=True)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('mentalhealth_ultimate_date.csv', index=False)


C:\Users\86153\AppData\Local\Temp\ipykernel_27340\1786485583.py:47: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


Progress: 996/1414 posts collected (70.44% complete)
Progress: 1989/1414 posts collected (140.66% complete)
Progress: 2982/1414 posts collected (210.89% complete)
Progress: 3982/1414 posts collected (281.61% complete)
Progress: 4976/1414 posts collected (351.91% complete)
Progress: 5976/1414 posts collected (422.63% complete)
Progress: 6249/1414 posts collected (441.94% complete)
Progress: 6274/1414 posts collected (443.71% complete)
Progress: 7267/1414 posts collected (513.93% complete)
Progress: 7292/1414 posts collected (515.70% complete)
                                                  title  \
0              r/MentalHealth is looking for moderators   
1                            Politics and Mental Health   
2     Be brutal, what would make you leave your frie...   
3     We'll be completely forgotten after a few gene...   
4                      My Place Is Right By Your Side 💕   
...                                                 ...   
7262  I thought i was doing okay, but I

In [16]:
import praw
import pandas as pd
import time
import random
from datetime import datetime
from prawcore.exceptions import RequestException, ResponseException, BadRequest

# Initialize the Reddit instance
reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx2043789012@'
)

# Fetch posts from different sorting categories and timeframes
subreddit = reddit.subreddit("suicidewatch")

# List to store post data
posts = []

# Set target post count between 1400 and 1600
desired_post_count = random.randint(1400, 1600)
batch_size = 1000

def fetch_posts(subreddit, sort_method, limit, time_filter=None):
    try:
        if sort_method == 'top' and time_filter:
            return subreddit.top(time_filter=time_filter, limit=limit)
        elif sort_method == 'controversial' and time_filter:
            return subreddit.controversial(time_filter=time_filter, limit=limit)
        elif sort_method == 'hot':
            return subreddit.hot(limit=limit)
        elif sort_method == 'new':
            return subreddit.new(limit=limit)
        elif sort_method == 'rising':
            return subreddit.rising(limit=limit)
        elif sort_method == 'random_rising':
            return subreddit.random_rising(limit=limit)
    except (RequestException, ResponseException, BadRequest) as e:
        print(f"API Error ({e}). Retrying after a 5-minute wait...")
        time.sleep(300)  # Wait for 300 seconds (5 minutes) before retrying
        return fetch_posts(subreddit, sort_method, limit, time_filter)

def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# Function to show progress
def show_progress(collected_posts, target_posts):
    percentage = (collected_posts / target_posts) * 100
    print(f"Progress: {collected_posts}/{target_posts} posts collected ({percentage:.2f}% complete)")

# Fetch posts in batches until we reach the target range
while len(posts) < desired_post_count:
    # Fetch a batch of 100 posts from 'hot'
    for submission in fetch_posts(subreddit, 'hot', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id  # Add post ID to help identify duplicates
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 posts from 'new'
    for submission in fetch_posts(subreddit, 'new', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'all time'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='all'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this year'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='year'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this month'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='month'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this week'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='week'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'today'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='day'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 rising posts
    for submission in fetch_posts(subreddit, 'rising', batch_size):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 controversial posts from 'this year'
    for submission in fetch_posts(subreddit, 'controversial', batch_size, time_filter='year'):
        posts.append({
            'title': submission.title,
            'body': submission.selftext,
            'author': submission.author.name if submission.author else 'deleted',
            'date': convert_timestamp(submission.created_utc),
            'id': submission.id
        })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 random rising posts (if supported)
    try:
        for submission in fetch_posts(subreddit, 'random_rising', batch_size):
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
        show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch
    except AttributeError:
        print("random_rising is not supported on this subreddit.")

# Only remove duplicates once we have collected enough posts
df = pd.DataFrame(posts)

# Remove duplicate posts based on the 'id' column
df.drop_duplicates(subset='id', keep='first', inplace=True)

# Drop the 'id' column since it's no longer needed
df.drop(columns=['id'], inplace=True)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('suicidewatch_ultimate_date.csv', index=False)


C:\Users\86153\AppData\Local\Temp\ipykernel_27340\4250697095.py:47: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


Progress: 947/1420 posts collected (66.69% complete)
Progress: 1929/1420 posts collected (135.85% complete)
Progress: 2921/1420 posts collected (205.70% complete)
Progress: 3919/1420 posts collected (275.99% complete)
Progress: 4914/1420 posts collected (346.06% complete)
Progress: 5912/1420 posts collected (416.34% complete)
Progress: 6272/1420 posts collected (441.69% complete)
Progress: 6297/1420 posts collected (443.45% complete)
Progress: 7294/1420 posts collected (513.66% complete)
Progress: 7319/1420 posts collected (515.42% complete)
                                                  title  \
0     New wiki on how to avoid accidentally encourag...   
1     Please remember that NO ACTIVISM of any kind i...   
2                    I'm bleeding out on a park bench.    
3     Hiding information about how to kill oneself s...   
4                                               RIP bro   
...                                                 ...   
7289  What exactly do they mean when th

In [14]:
import praw
import pandas as pd
import time
import random
from prawcore.exceptions import RequestException, ResponseException, BadRequest

# Initialize the Reddit instance
reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx2043789012@'
)

# Fetch posts from different sorting categories and timeframes
subreddit = reddit.subreddit("mentalhealth")

# List to store post data
posts = []

# Set target post count between 1400 and 1600
desired_post_count = random.randint(1400, 1600)
batch_size = 1000
desired_flair = "Need Support"  # Define the desired flair to filter posts

def fetch_posts(subreddit, sort_method, limit, time_filter=None):
    try:
        if sort_method == 'top' and time_filter:
            return subreddit.top(time_filter=time_filter, limit=limit)
        elif sort_method == 'controversial' and time_filter:
            return subreddit.controversial(time_filter=time_filter, limit=limit)
        elif sort_method == 'hot':
            return subreddit.hot(limit=limit)
        elif sort_method == 'new':
            return subreddit.new(limit=limit)
        elif sort_method == 'rising':
            return subreddit.rising(limit=limit)
        elif sort_method == 'random_rising':
            return subreddit.random_rising(limit=limit)
    except (RequestException, ResponseException, BadRequest) as e:
        print(f"API Error ({e}). Retrying after a 5-minute wait...")
        time.sleep(300)  # Wait for 300 seconds (5 minutes) before retrying
        return fetch_posts(subreddit, sort_method, limit, time_filter)

# Function to show progress
def show_progress(collected_posts, target_posts):
    percentage = (collected_posts / target_posts) * 100
    print(f"Progress: {collected_posts}/{target_posts} posts collected ({percentage:.2f}% complete)")

# Fetch posts in batches until we reach the target range
while len(posts) < desired_post_count:
    # Fetch a batch of 100 posts from 'hot'
    for submission in fetch_posts(subreddit, 'hot', batch_size):
        # Check if the post's flair matches the desired flair
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,  # Store the flair for reference
                'id': submission.id  # Add post ID to help identify duplicates
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 posts from 'new'
    for submission in fetch_posts(subreddit, 'new', batch_size):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'all time'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='all'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this year'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='year'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this month'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='month'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this week'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='week'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'today'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='day'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 rising posts
    for submission in fetch_posts(subreddit, 'rising', batch_size):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 controversial posts from 'this year'
    for submission in fetch_posts(subreddit, 'controversial', batch_size, time_filter='year'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 random rising posts (if supported)
    try:
        for submission in fetch_posts(subreddit, 'random_rising', batch_size):
            if submission.link_flair_text == desired_flair:
                posts.append({
                    'title': submission.title,
                    'body': submission.selftext,
                    'author': submission.author.name if submission.author else 'deleted',
                    'flair': submission.link_flair_text,
                    'id': submission.id
                })
        show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch
    except AttributeError:
        print("random_rising is not supported on this subreddit.")

# Only remove duplicates once we have collected enough posts
df = pd.DataFrame(posts)

# Remove duplicate posts based on the 'id' column
df.drop_duplicates(subset='id', keep='first', inplace=True)

# Drop the 'id' column since it's no longer needed
df.drop(columns=['id'], inplace=True)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('mentalhealth_nees_support.csv', index=False)


Progress: 196/1568 posts collected (12.50% complete)
Progress: 395/1568 posts collected (25.19% complete)
Progress: 438/1568 posts collected (27.93% complete)
Progress: 634/1568 posts collected (40.43% complete)
Progress: 842/1568 posts collected (53.70% complete)
Progress: 1071/1568 posts collected (68.30% complete)
Progress: 1119/1568 posts collected (71.36% complete)
Progress: 1126/1568 posts collected (71.81% complete)
Progress: 1335/1568 posts collected (85.14% complete)
Progress: 1342/1568 posts collected (85.59% complete)
Progress: 1538/1568 posts collected (98.09% complete)
Progress: 1737/1568 posts collected (110.78% complete)
Progress: 1780/1568 posts collected (113.52% complete)
Progress: 1976/1568 posts collected (126.02% complete)
Progress: 2184/1568 posts collected (139.29% complete)
Progress: 2413/1568 posts collected (153.89% complete)
Progress: 2465/1568 posts collected (157.21% complete)
Progress: 2472/1568 posts collected (157.65% complete)
Progress: 2681/1568 posts 

In [15]:
import praw
import pandas as pd
import time
import random
from datetime import datetime
from prawcore.exceptions import RequestException, ResponseException, BadRequest

# Initialize the Reddit instance
reddit = praw.Reddit(
    user_agent=True,
    client_id="kv4i1eHL9WnbMkNyS58xVw",
    client_secret="G6LzULvIFr_l3kC4yT80BlQ0Bim6rg",
    username="wucanyue1",
    password='Yhx2043789012@'
)

# Fetch posts from different sorting categories and timeframes
subreddit = reddit.subreddit("mentalhealth")

# List to store post data
posts = []

# Set target post count between 1400 and 1600
desired_post_count = random.randint(1400, 1600)
batch_size = 1000
desired_flair = "Need Support"  # Define the desired flair to filter posts

def fetch_posts(subreddit, sort_method, limit, time_filter=None):
    try:
        if sort_method == 'top' and time_filter:
            return subreddit.top(time_filter=time_filter, limit=limit)
        elif sort_method == 'controversial' and time_filter:
            return subreddit.controversial(time_filter=time_filter, limit=limit)
        elif sort_method == 'hot':
            return subreddit.hot(limit=limit)
        elif sort_method == 'new':
            return subreddit.new(limit=limit)
        elif sort_method == 'rising':
            return subreddit.rising(limit=limit)
        elif sort_method == 'random_rising':
            return subreddit.random_rising(limit=limit)
    except (RequestException, ResponseException, BadRequest) as e:
        print(f"API Error ({e}). Retrying after a 5-minute wait...")
        time.sleep(300)  # Wait for 300 seconds (5 minutes) before retrying
        return fetch_posts(subreddit, sort_method, limit, time_filter)

# Convert Unix timestamp to readable date format
def convert_timestamp(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# Function to show progress
def show_progress(collected_posts, target_posts):
    percentage = (collected_posts / target_posts) * 100
    print(f"Progress: {collected_posts}/{target_posts} posts collected ({percentage:.2f}% complete)")

# Fetch posts in batches until we reach the target range
while len(posts) < desired_post_count:
    # Fetch a batch of 100 posts from 'hot'
    for submission in fetch_posts(subreddit, 'hot', batch_size):
        # Check if the post's flair matches the desired flair
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,  # Store the flair for reference
                'date': convert_timestamp(submission.created_utc),  # Convert and store the date
                'id': submission.id  # Add post ID to help identify duplicates
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 posts from 'new'
    for submission in fetch_posts(subreddit, 'new', batch_size):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'all time'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='all'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this year'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='year'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this month'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='month'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'this week'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='week'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 top posts from 'today'
    for submission in fetch_posts(subreddit, 'top', batch_size, time_filter='day'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 rising posts
    for submission in fetch_posts(subreddit, 'rising', batch_size):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 controversial posts from 'this year'
    for submission in fetch_posts(subreddit, 'controversial', batch_size, time_filter='year'):
        if submission.link_flair_text == desired_flair:
            posts.append({
                'title': submission.title,
                'body': submission.selftext,
                'author': submission.author.name if submission.author else 'deleted',
                'flair': submission.link_flair_text,
                'date': convert_timestamp(submission.created_utc),
                'id': submission.id
            })
    show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch

    # Fetch a batch of 100 random rising posts (if supported)
    try:
        for submission in fetch_posts(subreddit, 'random_rising', batch_size):
            if submission.link_flair_text == desired_flair:
                posts.append({
                    'title': submission.title,
                    'body': submission.selftext,
                    'author': submission.author.name if submission.author else 'deleted',
                    'flair': submission.link_flair_text,
                    'date': convert_timestamp(submission.created_utc),
                    'id': submission.id
                })
        show_progress(len(posts), desired_post_count)  # Show progress after fetching a batch
    except AttributeError:
        print("random_rising is not supported on this subreddit.")

# Only remove duplicates once we have collected enough posts
df = pd.DataFrame(posts)

# Remove duplicate posts based on the 'id' column
df.drop_duplicates(subset='id', keep='first', inplace=True)

# Drop the 'id' column since it's no longer needed
df.drop(columns=['id'], inplace=True)

# Output the DataFrame
print(df)

# Save the DataFrame to a CSV file
df.to_csv('mentalhealth_need_support_ultimate.csv', index=False)


C:\Users\86153\AppData\Local\Temp\ipykernel_27340\2892366434.py:49: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')


Progress: 196/1511 posts collected (12.97% complete)
Progress: 395/1511 posts collected (26.14% complete)
Progress: 438/1511 posts collected (28.99% complete)
Progress: 634/1511 posts collected (41.96% complete)
Progress: 842/1511 posts collected (55.72% complete)
Progress: 1075/1511 posts collected (71.14% complete)
Progress: 1127/1511 posts collected (74.59% complete)
Progress: 1134/1511 posts collected (75.05% complete)
Progress: 1343/1511 posts collected (88.88% complete)
Progress: 1349/1511 posts collected (89.28% complete)
Progress: 1545/1511 posts collected (102.25% complete)
Progress: 1744/1511 posts collected (115.42% complete)
Progress: 1787/1511 posts collected (118.27% complete)
Progress: 1983/1511 posts collected (131.24% complete)
Progress: 2191/1511 posts collected (145.00% complete)
Progress: 2424/1511 posts collected (160.42% complete)
Progress: 2476/1511 posts collected (163.86% complete)
Progress: 2482/1511 posts collected (164.26% complete)
Progress: 2691/1511 posts

In [9]:
df.to_csv('mentalhealth4.csv', index=False)

In [13]:
import praw
import pandas as pd
import time
from datetime import datetime  # To convert Unix timestamp to a readable date

# Assuming you have already authenticated with Reddit using PRAW
# Example authentication (if not already done):
# reddit = praw.Reddit(client_id="YOUR_CLIENT_ID",
#                      client_secret="YOUR_CLIENT_SECRET",
#                      username="YOUR_REDDIT_USERNAME",
#                      password="YOUR_REDDIT_PASSWORD",
#                      user_agent="YOUR_USER_AGENT")

subreddit = reddit.subreddit("SuicideWatch")
comments = []

# Loop until you gather data from posts
while len(comments) < 5000:
    for submission in subreddit.hot(limit=100):
        try:
            # Convert the post creation time from Unix timestamp to human-readable format
            post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')

            # Append the relevant information to the comments list
            comments.append({
                'title': submission.title, 
                'content': submission.selftext,  # Post content (selftext)
                'user': submission.author.name if submission.author else '[deleted]',  # Handle deleted authors
                'date': post_date  # Post creation date
            })

            if len(comments) >= 5000:  # Stop once we have 5000 records
                break

        except praw.exceptions.PRAWException as e:
            if e.response.status_code == 429:  # Handle TooManyRequests error
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)  # Wait for 60 seconds before retrying
                break  # Break the inner loop to retry fetching submissions
            else:
                print(f"An error occurred: {e}")  # Handle other exceptions
        except AttributeError:  # Handle case when author is deleted or unavailable
            post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            comments.append({
                'title': submission.title, 
                'content': submission.selftext, 
                'user': '[deleted]',  # Assign '[deleted]' for unavailable authors
                'date': post_date  # Post creation date
            })
    time.sleep(2)  # Add a delay of 2 seconds between requests

# Convert the list to a DataFrame
df = pd.DataFrame(comments)

# Save the DataFrame to a CSV file
df.to_csv('reddit_suicidewatch_posts.csv', index=False)

print("Data collection complete and saved to reddit_suicidewatch_posts.csv")


C:\Users\86153\AppData\Local\Temp\ipykernel_14992\3035103538.py:22: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')


Data collection complete and saved to reddit_suicidewatch_posts.csv


In [ ]:
import praw
import pandas as pd
import time
from datetime import datetime  # To convert Unix timestamp to a readable date

# Assuming you have already authenticated with Reddit using PRAW
# Example authentication (if not already done):
# reddit = praw.Reddit(client_id="YOUR_CLIENT_ID",
#                      client_secret="YOUR_CLIENT_SECRET",
#                      username="YOUR_REDDIT_USERNAME",
#                      password="YOUR_REDDIT_PASSWORD",
#                      user_agent="YOUR_USER_AGENT")

# Subreddit 'mentalhealth' and flair 'Need Support' URL-specific posts
subreddit = reddit.subreddit("mentalhealth")
comments = []

# Loop until you gather data from posts with the flair "Need Support"
while len(comments) < 5000:
    for submission in subreddit.search('flair:"Need Support"', sort='hot', limit=100):
        try:
            # Convert the post creation time from Unix timestamp to human-readable format
            post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')

            comments.append({
                'title': submission.title, 
                'content': submission.selftext,  # Post content (selftext)
                'user': submission.author.name if submission.author else '[deleted]',  # Handle deleted authors
                'date': post_date  # Post creation date
            })

            if len(comments) >= 5000:  # Stop once we have 5000 records
                break

        except praw.exceptions.PRAWException as e:
            if e.response.status_code == 429:  # Handle TooManyRequests error
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)  # Wait for 60 seconds before retrying
                break  # Break the inner loop to retry fetching submissions
            else:
                print(f"An error occurred: {e}")  # Handle other exceptions
        except AttributeError:  # Handle case when author is deleted or unavailable
            post_date = datetime.utcfromtimestamp(submission.created_utc).strftime('%Y-%m-%d %H:%M:%S')
            comments.append({
                'title': submission.title, 
                'content': submission.selftext, 
                'user': '[deleted]',  # Assign '[deleted]' for unavailable authors
                'date': post_date  # Post creation date
            })
    time.sleep(2)  # Add a delay of 2 seconds between requests

# Convert the list to a DataFrame
df = pd.DataFrame(comments)

# Save the DataFrame to a CSV file
df.to_csv('reddit_need_support_posts_filtered.csv', index=False)

print("Data collection complete and saved to reddit_need_support_posts_filtered.csv")


In [14]:
import praw
import pandas as pd
import time

# Initialize PRAW Reddit object (make sure to authenticate appropriately)
# Example: reddit = praw.Reddit(client_id='YOUR_CLIENT_ID', client_secret='YOUR_CLIENT_SECRET', user_agent='YOUR_USER_AGENT')

subreddit = reddit.subreddit("mentalhealth")
comments = []
target_post_count = 1000  # Set the target number of posts to scrape

# Loop until you gather data from 1000 posts
while len(comments) < target_post_count:
    for submission in subreddit.hot(limit=100):  # Fetch top 100 posts in 'hot' category
        try:
            comments.append({
                'title': submission.title, 
                'content': submission.selftext,  # Post content (selftext)
                'user': submission.author.name if submission.author else '[deleted]'  # Handle deleted authors
            })

            # Notify progress after each post is scraped
            print(f"Scraped post {len(comments)}: {submission.title}")

            if len(comments) >= target_post_count:  # Stop once we have 1000 records
                break
        except praw.exceptions.PRAWException as e:
            if e.response.status_code == 429:  # Handle TooManyRequests error
                print("Rate limit exceeded. Sleeping for 60 seconds...")
                time.sleep(60)  # Wait for 60 seconds before retrying
                break  # Break the inner loop to retry fetching submissions
            else:
                print(f"An error occurred: {e}")  # Handle other exceptions
        except AttributeError:  # Handle case when author is deleted or unavailable
            comments.append({
                'title': submission.title, 
                'content': submission.selftext, 
                'user': '[deleted]'  # Assign '[deleted]' for unavailable authors
            })
            print(f"Scraped post {len(comments)}: {submission.title} (Author: [deleted])")
            
    time.sleep(2)  # Add a delay of 2 seconds between requests

# Convert the list to a DataFrame
df = pd.DataFrame(comments)

# Save the DataFrame to a CSV file
df.to_csv('reddit_posts.csv', index=False)

print("Data collection complete and saved to reddit_posts.csv")


ResponseException: received 401 HTTP response